In [49]:
import pandas as pd

In [75]:
import os
from glob import glob
PATH = os.getcwd()
os.path.join(PATH, "Fantasy-Premier-League\data\2016-17\players")
EXT = "*.csv"
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT)) if "\gw.csv" in file]

In [ ]:
for file in all_csv_files:
    

In [204]:
data = []
for file in all_csv_files:
    dataframe = pd.read_csv(file)
    
#     temp = []
#     print(dataframe.size)
    for i in range(len(dataframe.index) - 10):
        temp = []
        for j in range(i, i + 10):
            temp.append(dataframe.iloc[j][
                                        ['assists',
                                         'bonus',
                                         'bps',
                                         'clean_sheets',
                                         'creativity',
                                         'goals_conceded',
                                         'goals_scored',
                                         'ict_index',
                                         'influence',
                                         'minutes',
                                         'opponent_team',
                                         'own_goals',
                                         'penalties_missed',
                                         'penalties_saved',
                                         'red_cards',
                                         'saves',
                                         'selected',
                                         'team_a_score',
                                         'team_h_score',
                                         'threat',
                                         'transfers_balance',
                                         'transfers_in',
                                         'transfers_out',
                                         'value',
                                         'was_home',
                                         'yellow_cards']
                                         ])
            temp[len(temp) - 1]["was_home"] = int(temp[len(temp) - 1]["was_home"])
        temp2 = [temp, dataframe.iloc[i+10]["total_points"]]
        data.append(temp2)
#         break
#     break


In [206]:
len(data)

17009

In [207]:
len(data[0][0][0])

26

In [208]:
# data is total data
# data[i] denotes one input unit which consists of a pair [[10, 26], 1] where first item of the pair is a sequence length 10 
# denting 10 weeks of data and second item is the 11th week points 

In [209]:
hmm = [[[[1, 2]]], 3]

import pickle

In [215]:
# len(data[0][0][0])

26

In [216]:
file = open("2016-17_pps.dat", "wb")
pickle.dump(data, file)
file.close()

In [218]:
objects = []
with (open("2016-17_pps.dat", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [217]:
print("F"
     )

F


In [219]:
len(objects)

1

In [220]:
len(objects[0])

17009

In [258]:
trainX = []
trainY = []

In [259]:
for i in data:
    cur = i[0]
    cur = pd.DataFrame(cur)
    cur = cur.astype(float)
    trainX.append(cur)
    cur2 = i[1]
    cur2 = pd.DataFrame([cur2])
    cur2 = cur2.astype(float)
    trainY.append(cur2)

In [234]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [265]:
trainX = np.array(trainX)
trainY = np.array(trainY)

In [260]:
trainXdf = pd.DataFrame(trainX)
trainYdf = pd.DataFrame(trainY)

ValueError: Must pass 2-d input. shape=(17009, 10, 26)

0
<class 'int'>


AttributeError: 'int' object has no attribute 'astype'

In [262]:
scaler = StandardScaler()
scalerX = scaler.fit(trainX)
trainXdf_scaled = scalerX.transform(trainX)
scalerY = scaler.fit(trainY)
trainYdf_scaled = scalerY.transform(trainY)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [270]:
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
# scaler2  = StandardScaler()
# trainY = scaler2.fit_transform(trainY.reshape(-1, trainY.shape[-1])).reshape(trainY.shape)

In [271]:
trainX

array([[[-0.18865477, -0.21732484, -0.6157846 , ...,  0.25864099,
         -1.00009407, -0.25166442],
        [-0.18865477, -0.21732484, -0.6157846 , ...,  0.25864099,
          0.99990594, -0.25166442],
        [-0.18865477, -0.21732484, -0.6157846 , ...,  0.17988962,
         -1.00009407, -0.25166442],
        ...,
        [ 4.55252291, -0.21732484,  1.67388623, ...,  0.10113826,
         -1.00009407, -0.25166442],
        [-0.18865477, -0.21732484, -0.6157846 , ...,  0.10113826,
          0.99990594, -0.25166442],
        [-0.18865477, -0.21732484,  0.57882627, ...,  0.10113826,
         -1.00009407, -0.25166442]],

       [[-0.18865477, -0.21732484, -0.6157846 , ...,  0.25864099,
          0.99990594, -0.25166442],
        [-0.18865477, -0.21732484, -0.6157846 , ...,  0.17988962,
         -1.00009407, -0.25166442],
        [-0.18865477, -0.21732484, -0.6157846 , ...,  0.17988962,
          0.99990594, -0.25166442],
        ...,
        [-0.18865477, -0.21732484, -0.6157846 , ...,  

In [277]:
trainY = trainY.reshape([17009, 1])

In [280]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            23296     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 35,745
Trainable params: 35,745
Non-trainable params: 0
_________________________________________________________________


In [284]:
history = model.fit(trainX, trainY, epochs=50, batch_size=16, validation_split=0.1, verbose=1)


Epoch 1/50
957/957 [==============================] - 8s 9ms/step - loss: 0.6599 - val_loss: 0.6962
Epoch 2/50
957/957 [==============================] - 8s 9ms/step - loss: 0.6470 - val_loss: 0.6987
Epoch 3/50
957/957 [==============================] - 8s 8ms/step - loss: 0.6297 - val_loss: 0.7062
Epoch 4/50
957/957 [==============================] - 8s 8ms/step - loss: 0.6053 - val_loss: 0.7653
Epoch 5/50
957/957 [==============================] - 8s 9ms/step - loss: 0.5793 - val_loss: 0.7131
Epoch 6/50
957/957 [==============================] - 8s 8ms/step - loss: 0.5661 - val_loss: 0.7808
Epoch 7/50
957/957 [==============================] - 8s 9ms/step - loss: 0.5446 - val_loss: 0.7381
Epoch 8/50
957/957 [==============================] - 8s 9ms/step - loss: 0.5050 - val_loss: 0.7646
Epoch 9/50
957/957 [==============================] - 8s 9ms/step - loss: 0.4887 - val_loss: 0.7597
Epoch 10/50
957/957 [==============================] - 8s 9ms/step - loss: 0.4565 - val_loss: 0.7881

In [423]:
testX = pd.read_csv("test.csv")


In [441]:
testX = testX[ ['assists',
                                         'bonus',
                                         'bps',
                                         'clean_sheets',
                                         'creativity',
                                         'goals_conceded',
                                         'goals_scored',
                                         'ict_index',
                                         'influence',
                                         'minutes',
                                         'opponent_team',
                                         'own_goals',
                                         'penalties_missed',
                                         'penalties_saved',
                                         'red_cards',
                                         'saves',
                                         'selected',
                                         'team_a_score',
                                         'team_h_score',
                                         'threat',
                                         'transfers_balance',
                                         'transfers_in',
                                         'transfers_out',
                                         'value',
                                         'was_home',
                                         'yellow_cards']]

In [442]:
testX["was_home"] = testX["was_home"].astype(int)

In [443]:
testXnp = np.asarray(testX)

In [444]:
testXnp.shape

(38, 26)

In [446]:
testXnp = testXnp[0:10].reshape([1, 10, 26])

In [447]:
# testXnp = scaler.transform(testXnp)
testXnp = scaler.transform(testXnp.reshape(-1, testXnp.shape[-1])).reshape(testXnp.shape)


In [448]:
testXnp.shape

(1, 10, 26)

In [437]:
testXnp

array([[ 4.64155664e-16, -7.56635366e-16,  3.00000000e+00,
         8.60740330e-15,  6.00000000e-01,  2.00000000e+00,
        -2.22009919e-14,  1.90000000e+00,  4.00000000e-01,
         9.00000000e+00,  1.20000000e+01, -3.17210737e-15,
         3.60606092e-15,  1.06881322e-15,  3.36508516e-15,
        -1.25778918e-14,  2.51360000e+04,  2.84156064e-15,
         4.00000000e+00,  1.80000000e+01,  2.73596968e-17,
         2.10479693e-16, -5.99908153e-17,  5.00000000e+01,
         8.22145965e-16, -1.72971918e-14],
       [ 4.64155664e-16, -7.56635366e-16,  1.60000000e+01,
         8.60740330e-15,  5.29000000e+01,  3.00000000e+00,
        -2.22009919e-14,  7.40000000e+00,  2.12000000e+01,
         9.00000000e+01,  1.40000000e+01, -3.17210737e-15,
         3.60606092e-15,  1.06881322e-15,  3.36508516e-15,
        -1.25778918e-14,  2.78240000e+04,  2.00000000e+00,
         3.00000000e+00,  1.39914941e-14, -1.51500000e+03,
         9.45000000e+02,  2.46000000e+03,  5.00000000e+01,
         8.22

In [449]:
testXnp = testXnp.reshape([1, 10, 26])

In [417]:
# type(trainX)

print(scaler2.inverse_transform(trainY[6523].reshape((-1, 1))))

[[24.]]


In [418]:
hmm = trainX[6523].reshape([1, 10, 26])

In [450]:
prediction = model.predict(testXnp) #shape = (n, 1) where n is the n_days_for_prediction


1/1 [==============================] - 0s 30ms/step


In [451]:
prediction = scaler2.inverse_transform(prediction)

In [452]:
prediction

array([[1307.6477]], dtype=float32)

In [416]:
np.where(scaler2.inverse_transform(trainY.reshape((-1, 1))) == 24)[0]


array([6523], dtype=int64)